### Chin-Ting Ko 02/09/2018 <br> Data Engineer Challenge###

# Part 1: ETL Pipeline and SQL

In [309]:
import pandas as pd
df = pd.read_csv('data.csv')
df.describe()

<bound method DataFrame.describe of       InvoiceNo  Customer_ID                 Date       Planet  \
0        536365        17850          Dec 01 2010       Abydos   
1        536366        17850                40513       Abydos   
2        536367        13047           12/01/2010       Abydos   
3        536368        13047          Dec 01 2010       Abydos   
4        536369        13047           12/01/2010       Abydos   
5        536370        12583                40513    Altair IV   
6        536371        13748  2010-12-01 09:00:00       Abydos   
7        536372        17850                40513       Abydos   
8        536373        17850          Dec 01 2010       Abydos   
9        536374        15100                40513       Abydos   
10       536375        17850                40513       Abydos   
11       536376        15291          Dec 01 2010       Abydos   
12       536377        17850          Dec 01 2010       Abydos   
13       536378        14688  2010-12-01

In [310]:
print df['Purchase'][1]

[{'ItemNo': 22633, 'UnitPrice': 1.85, 'Description': u'BRAWNDO', 'Quantity': 6}, {'ItemNo': 22632, 'UnitPrice': 1.85, 'Description': u'HAND WARMER RED POLKA DOT', 'Quantity': 6}]


In [311]:
import ast
df['Purchase'] = df['Purchase'].apply(ast.literal_eval)

In [312]:
df['Purchase']= df['Purchase'].apply(lambda x: json_normalize(x))

In [313]:
print df['Purchase'][0]

                           Description  ItemNo  Quantity  UnitPrice
0                     ADORIAN CROSSBOW  85123A         6       2.55
1                  WHITE METAL LANTERN   71053         6       3.39
2       CREAM CUPID HEARTS COAT HANGER  84406B         8       2.75
3  KNITTED UNION FLAG HOT WATER BOTTLE  84029G         6       3.39
4       RED WOOLLY HOTTIE WHITE HEART.  84029E         6       3.39
5         SET 7 BABUSHKA NESTING BOXES   22752         2       7.65
6    GLASS STAR FROSTED T-LIGHT HOLDER   21730         6       4.25


In [314]:
print df.iloc[0]

InvoiceNo                                                 536365
Customer_ID                                                17850
Date                                                 Dec 01 2010
Planet                                                    Abydos
Purchase                                  Description  ItemNo...
Name: 0, dtype: object


In [315]:
print df['Purchase'][:5]

0                               Description  ItemNo...
1                     Description  ItemNo  Quantity...
2                               Description  ItemNo...
3                          Description  ItemNo  Qua...
4                    Description  ItemNo  Quantity ...
Name: Purchase, dtype: object


In [316]:
null_data = df[df['Customer_ID'].isnull()]
print len(null_data)

2173


In [317]:
null_data = df[df['Purchase'].isnull()]
print len(null_data)

0


In [318]:
print df['Date'].apply(lambda x: type(x)) [:5]

0    <type 'str'>
1    <type 'str'>
2    <type 'str'>
3    <type 'str'>
4    <type 'str'>
Name: Date, dtype: object


In [319]:
from datetime import datetime  # datetime cleaning
df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x, "%b %d %Y").strftime("%Y-%m-%d") if str(x)[3]==' ' else x)

In [320]:
df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y").strftime("%Y-%m-%d") if str(x)[2]=='/' else x)

In [321]:
df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d") if len(str(x))==19 else x)

In [322]:
import numpy as np
df['Date'] = df['Date'].apply(lambda x: np.nan if len(str(x))==5 else x)

In [323]:
df['Date'] = df['Date'].fillna(method='bfill')
print df['Date'] [:5]

0    2010-12-01
1    2010-12-01
2    2010-12-01
3    2010-12-01
4    2010-12-01
Name: Date, dtype: object


In [324]:
df.to_csv('output.csv')

### output.csv is cleaned csv file

In [326]:
print df.iloc[:3]

  InvoiceNo  Customer_ID        Date  Planet  \
0    536365        17850  2010-12-01  Abydos   
1    536366        17850  2010-12-01  Abydos   
2    536367        13047  2010-12-01  Abydos   

                                            Purchase  
0                             Description  ItemNo...  
1                   Description  ItemNo  Quantity...  
2                             Description  ItemNo...  


In [485]:
df['Customer_ID'] = df['Customer_ID'].astype(str).apply(lambda x: x[:-2])

In [486]:
df2 = df.set_index(['InvoiceNo', 'Customer_ID', 'Date', 'Planet'])

In [487]:
print df2.index[1]

('536366', '17850', '2010-12-01', 'Abydos')


In [488]:
for index, row in df2.iloc[:1].iterrows():  # test SQL insert script
    for x in row['Purchase'].iterrows():
        print 'INSERT INTO Orders (InvoiceNo, Customer_ID, Date, Planet, Description, ItemNo, Quantity, UnitPrice) VALUES ('+str(index)[1:-1]+", '"+ str(x[1]['Description'])+"', '"+ str(x[1]['ItemNo'])+"', "+ str(x[1]['Quantity'])+", "+ str(x[1]['UnitPrice'])+');'

INSERT INTO Orders (InvoiceNo, Customer_ID, Date, Planet, Description, ItemNo, Quantity, UnitPrice) VALUES ('536365', '17850', '2010-12-01', 'Abydos', 'ADORIAN CROSSBOW', '85123A', 6, 2.55);
INSERT INTO Orders (InvoiceNo, Customer_ID, Date, Planet, Description, ItemNo, Quantity, UnitPrice) VALUES ('536365', '17850', '2010-12-01', 'Abydos', 'WHITE METAL LANTERN', '71053', 6, 3.39);
INSERT INTO Orders (InvoiceNo, Customer_ID, Date, Planet, Description, ItemNo, Quantity, UnitPrice) VALUES ('536365', '17850', '2010-12-01', 'Abydos', 'CREAM CUPID HEARTS COAT HANGER', '84406B', 8, 2.75);
INSERT INTO Orders (InvoiceNo, Customer_ID, Date, Planet, Description, ItemNo, Quantity, UnitPrice) VALUES ('536365', '17850', '2010-12-01', 'Abydos', 'KNITTED UNION FLAG HOT WATER BOTTLE', '84029G', 6, 3.39);
INSERT INTO Orders (InvoiceNo, Customer_ID, Date, Planet, Description, ItemNo, Quantity, UnitPrice) VALUES ('536365', '17850', '2010-12-01', 'Abydos', 'RED WOOLLY HOTTIE WHITE HEART.', '84029E', 6, 3.3

In [501]:
for index, row in df.iloc[:10].iterrows():
    print "INSERT INTO Orders (InvoiceNo, Customer_ID, Date, Planet) VALUES ('"+ row['InvoiceNo']+"', '"+ row['Customer_ID']+"', '"+ row['Date']+"', '"+ row['Planet']+"');"

INSERT INTO Orders (InvoiceNo, Customer_ID, Date, Planet) VALUES ('536365', '17850', '2010-12-01', 'Abydos');
INSERT INTO Orders (InvoiceNo, Customer_ID, Date, Planet) VALUES ('536366', '17850', '2010-12-01', 'Abydos');
INSERT INTO Orders (InvoiceNo, Customer_ID, Date, Planet) VALUES ('536367', '13047', '2010-12-01', 'Abydos');
INSERT INTO Orders (InvoiceNo, Customer_ID, Date, Planet) VALUES ('536368', '13047', '2010-12-01', 'Abydos');
INSERT INTO Orders (InvoiceNo, Customer_ID, Date, Planet) VALUES ('536369', '13047', '2010-12-01', 'Abydos');
INSERT INTO Orders (InvoiceNo, Customer_ID, Date, Planet) VALUES ('536370', '12583', '2010-12-01', 'Altair IV');
INSERT INTO Orders (InvoiceNo, Customer_ID, Date, Planet) VALUES ('536371', '13748', '2010-12-01', 'Abydos');
INSERT INTO Orders (InvoiceNo, Customer_ID, Date, Planet) VALUES ('536372', '17850', '2010-12-01', 'Abydos');
INSERT INTO Orders (InvoiceNo, Customer_ID, Date, Planet) VALUES ('536373', '17850', '2010-12-01', 'Abydos');
INSERT 

In [506]:
for index, row in df2.iloc[:1].iterrows():
    for x in row['Purchase'].iterrows():
        print "INSERT INTO Products_Orders (ItemNo, Quantity, UnitPrice) VALUES ('"+ str(x[1]['ItemNo'])+"', "+ str(x[1]['Quantity'])+", "+ str(x[1]['UnitPrice'])+");"
        

INSERT INTO Products_Orders (ItemNo, Quantity, UnitPrice) VALUES ('85123A', 6, 2.55);
INSERT INTO Products_Orders (ItemNo, Quantity, UnitPrice) VALUES ('71053', 6, 3.39);
INSERT INTO Products_Orders (ItemNo, Quantity, UnitPrice) VALUES ('84406B', 8, 2.75);
INSERT INTO Products_Orders (ItemNo, Quantity, UnitPrice) VALUES ('84029G', 6, 3.39);
INSERT INTO Products_Orders (ItemNo, Quantity, UnitPrice) VALUES ('84029E', 6, 3.39);
INSERT INTO Products_Orders (ItemNo, Quantity, UnitPrice) VALUES ('22752', 2, 7.65);
INSERT INTO Products_Orders (ItemNo, Quantity, UnitPrice) VALUES ('21730', 6, 4.25);


In [511]:
products = []
for index, row in df2.iloc[:2].iterrows(): # only return unique product list
    for x in row['Purchase'].iterrows():
        if str(x[1]['ItemNo']) not in products:
            products.append(str(x[1]['ItemNo']))
            print "INSERT INTO Products (ItemNo, Description) VALUES ('"+ str(x[1]['ItemNo'])+"', '"+ str(x[1]['Description'])+"';"
        

INSERT INTO Products (ItemNo, Description) VALUES ('85123A', 'ADORIAN CROSSBOW';
INSERT INTO Products (ItemNo, Description) VALUES ('71053', 'WHITE METAL LANTERN';
INSERT INTO Products (ItemNo, Description) VALUES ('84406B', 'CREAM CUPID HEARTS COAT HANGER';
INSERT INTO Products (ItemNo, Description) VALUES ('84029G', 'KNITTED UNION FLAG HOT WATER BOTTLE';
INSERT INTO Products (ItemNo, Description) VALUES ('84029E', 'RED WOOLLY HOTTIE WHITE HEART.';
INSERT INTO Products (ItemNo, Description) VALUES ('22752', 'SET 7 BABUSHKA NESTING BOXES';
INSERT INTO Products (ItemNo, Description) VALUES ('21730', 'GLASS STAR FROSTED T-LIGHT HOLDER';
INSERT INTO Products (ItemNo, Description) VALUES ('22633', 'BRAWNDO';
INSERT INTO Products (ItemNo, Description) VALUES ('22632', 'HAND WARMER RED POLKA DOT';


### Please give us the SQL code you write to create the database/tables.
**I only created three tables: orders, products, and products_orders (for many to many relationship)
<br>I didn't create customer and planet tables, to avoid too many joins**

CREATE TABLE `Products_Orders`.`Orders` (<br>
  `idOrders` INT NOT NULL,<br>
  `InvoiceNo` VARCHAR(45) NULL,<br>
  `Customer_ID` VARCHAR(45) NULL,<br>
  `Dates` VARCHAR(45) NULL,<br>
  `Planet` VARCHAR(45) NULL,<br>
  PRIMARY KEY (`idOrders`));<br>
  <br>
ALTER TABLE `Products_Orders`.`Orders`<br> 
CHANGE COLUMN `idOrders` `idOrders` INT(11) NOT NULL AUTO_INCREMENT ;<br>

CREATE TABLE `Products_Orders`.`Products` (<br>
  `idProducts` INT NOT NULL,<br>
  `ItemNo` VARCHAR(45) NULL,<br>
  `Description` VARCHAR(45) NULL,<br>
  PRIMARY KEY (`idProducts`));<br>
<br>
ALTER TABLE `Products_Orders`.`Poducts` <br>
CHANGE COLUMN `idProducts` `idProducts` INT(11) NOT NULL AUTO_INCREMENT ;<br>
<br>
CREATE TABLE `Products_Orders`.`Products_Orders` (<br>
  `idProducts_Orders` INT NOT NULL,<br>
  `ItemNo` VARCHAR(45) NULL,<br>
  `Quantity` VARCHAR(45) NULL,<br>
   `UnitPrice` VARCHAR(45) NULL,<br>
  PRIMARY KEY (`idProducts_Orders`));<br>
  <br>
ALTER TABLE `Products_Orders`.`Products_Orders` <br>
CHANGE COLUMN `idProducts_Orders` `idProducts_Orders` INT(11) NOT NULL AUTO_INCREMENT ;<br>

In [522]:
file1 = open("insert_orders.txt","w") # final insert script txt file for ORDER table

for index, row in df.iterrows():
    file1.write("INSERT INTO Products_Orders.Orders(InvoiceNo, Customer_ID, Dates, Planet) VALUES ('"+ row['InvoiceNo']+"', '"+ row['Customer_ID']+"', '"+ row['Date']+"', '"+ row['Planet']+"');\n")
 
file1.close()

In [528]:
from django.utils.encoding import smart_str, smart_unicode 
file2 = open("insert_products_orders.txt","w")  # final insert script txt file for PRODUCT_ORDER table

for index, row in df2.iterrows():
    for x in row['Purchase'].iterrows():
        file2.write("INSERT INTO Products_Orders.Products_Orders(ItemNo, Quantity, UnitPrice, InvoiceNo) VALUES ('"+ smart_str(x[1]['ItemNo'])+"', "+ smart_str(x[1]['Quantity'])+", "+ smart_str(x[1]['UnitPrice'])+", "+ str(index)[1:9]+");\n")
        
file2.close()

In [525]:
file3 = open("insert_products.txt","w") 

products = []
for index, row in df2.iterrows():  # final insert script txt file for PRODUCT table
    for x in row['Purchase'].iterrows():
        if smart_str(x[1]['ItemNo']) not in products:
            products.append(smart_str(x[1]['ItemNo']))
            file3.write("INSERT INTO Products_Orders.Products(ItemNo, Description) VALUES ('"+ smart_str(x[1]['ItemNo'])+"', '"+ smart_str(x[1]['Description']).replace("'","")+"');\n")
        
file3.close()

Please write SQL queries to answer the following questions. Give us both your SQL query and your answer. Specify any assumptions you make.

### 1. How many unique customers does the Bebop have?

SELECT COUNT(DISTINCT Customer_ID) FROM PRODUCTS_ORDERS.ORDERS;**<br>
<br>
COUNT(DISTINCT Customer_ID)<br>
  4373<br>


### 2. Which customer is the second biggest customer by revenue, excluding cancellations?

SELECT O.Customer_ID, SUM(P_O.Quantity * P_O.UnitPrice) AS Revenue FROM PRODUCTS_ORDERS.PRODUCTS_ORDERS AS P_O 
<br>JOIN PRODUCTS_ORDERS.ORDERS AS O ON P_O.InvoiceNo = O.InvoiceNo GROUP BY O.Customer_ID<br> ORDER BY Revenue DESC LIMIT 3;<br>
<br>
Customer_ID, Revenue<br>
n, 1744214.579740148 <-- n is Null Customer_ID<br>
18102, 259657.2984648943 <-- Top 1<br>
14646, 280206.0194363296 <-- Top 2<br>

### 3. How many customers have visited the Bebop on multiple planets?

SELECT COUNT(TEMP.Customer_ID) FROM (SELECT Customer_ID, COUNT(planet) FROM PRODUCTS_ORDERS.ORDERS <br>GROUP BY Customer_ID HAVING COUNT(planet) > 1 )AS TEMP;<br>
<br>
COUNT(TEMP.Customer_ID)<br>
3060<br>


### 4. Which product has had the most price changes?

<br>SELECT ItemNo, MAX(UnitPrice) - MIN(UnitPrice) AS PriceChange FROM PRODUCTS_ORDERS.PRODUCTS_ORDERS <br>GROUP By ItemNo ORDER BY PriceChange DESC LIMIT 1;<br>
<br>
ItemNo, PriceChange<br>
AMAZONFEE, 13321.570083618164<br>


### 5. Which item attracts the most new customers?

SELECT ItemNo, SUM(Quantity) FROM PRODUCTS_ORDERS.PRODUCTS_ORDERS GROUP BY ItemNo ORDER BY SUM(Quantity) DESC;<br>
<br>
ItemNo, SUM(Quantity)<br>
  23843, 80995<br>


### 6. Which customer has the shortest average length of time between purchases?

SELECT Customer_ID, MAX(Dates) - MIN(Dates)/ COUNT(Dates) FROM PRODUCTS_ORDERS.ORDERS GROUP BY Customer_ID;<br>
<br>
Customer_ID, LengthOfTime, Count(Dates), Min(Dates), MAX(Dates)<br>
18176, 1005, 2, 2010-12-16, 2010-12-21<br>
13370, 1005, 2, 2010-12-03, 2010-12-08<br>
15880, 1005, 2, 2010-12-05, 2010-12-05<br>
13295, 1005, 2, 2010-12-02, 2010-12-15<br>
15922, 1005, 2, 2010-12-01, 2010-12-02<br>
14813, 1005, 2, 2010-12-05, 2010-12-05<br>
15221, 1005, 2, 2010-12-05, 2010-12-08<br>
16781, 1005, 2, 2010-12-02, 2010-12-09<br>
17691, 1005, 2, 2010-12-06, 2010-12-16<br>
18269, 1005, 2, 2010-12-08, 2010-12-16<br>
16858, 1005, 2, 2010-12-06, 2010-12-08<br>
13369, 1005, 2, 2010-12-09, 2010-12-20<br>
15279, 1005, 2, 2010-12-09, 2010-12-20<br>


### 7. For each invoice, calculate the total revenue for each item and return the combined revenue of the top 3 items by revenue. You can ignore invoices that have fewer than 3 items.

SELECT P_O1.InvoiceNo, SUM(Quantity * UnitPrice) <br>
FROM PRODUCTS_ORDERS.PRODUCTS_ORDERS AS P_O1 <br>
WHERE 3 ><br>
(SELECT COUNT(Quantity * UnitPrice) <br>
FROM PRODUCTS_ORDERS.PRODUCTS_ORDERS AS P_O2<br>
WHERE P_O2.Quantity * UnitPrice > P_O1.Quantity * UnitPrice <br>
and P_O1.InvoiceNo = P_O2.InvoiceNo)<br>
GROUP BY InvoiceNo ORDER BY InvoiceNo<br>
<br>
InvoiceNo, SUM(Quantity * UnitPrice)<br>
536365, 123.82000160217285<br>
536366, 22.200000286102295<br>
536367, 84.08000183105469<br>
536368, 70.04999828338623<br>
536369, 17.84999942779541<br>
536370, 64.2000002861023<br>
536371, 203.99999618530273<br>
536372, 22.200000286102295<br>
536373, 206.85999822616577<br>
...<br>


# Part 2: MongoDB

Please write MongoDB queries to do the following. For questions 2 and 3, please provide only a sample query. Please give us your MongoDB scripts.

### 1. Create a collection for the transaction log and insert one example invoice as a document into the collection. You do not need to load the entire collection.

db.createCollection("Products_Orders") <br>

db.Products_Orders.insert(<br>
   { InvoiceNo : '536365', <br>
     Customer_ID : '17850', <br>
     Date : '2010-12-01', <br>
     Planet : 'Abydos'<br>
     Purchase: { ItemNo: '85123A',<br>
                 UnitPrice: 2.55,<br>
                 Description: 'ADORIAN CROSSBOW',<br>
                 Quantity: 6<br>
     }
   } 
)

### 2. How many unique weapons does the Bebop have?

db.collection.distinct('Purchase'.'ItemNo').length<br>
3958<br>

### 3. Which customer is the second biggest customer by revenue, excluding cancellations?

db.Products_Orders.aggregate(<br>
    {"\$group": <br>
         \{_id: "\$Customer_ID"},<br>
           revenue : {\$sum : "\$Purchase"."\$Quantity" * "\$Purchase"."\$UnitPrice"}}, <br>
    {\$sort: {"revenue": -1}}, <br>
    {\$limit: 2} <br>
)<br>
<br>
Customer_ID, Revenue <br>
14646, 280206.0194363296 <-- Top 2<br>

# Part 3: Runtime Analysis in Python

### 1. In plain English, explain what this function does.

This function (mystery) is to return list of duplicate values in input array.

### 2. Describe the runtime of this code as a function of the length of the input array. Please explain your answer. (Bonus points for showing that your bound is tight.)

N: length of input array
Runtime is O(N^2), since there are two nested for loop.
For 1st loop i iterates from 0~N, and 2nd loop j also iterates from 0~N
so O(N) * O(N) = O(N^2)

### 3. Write a more efficient implementation of this function. Please do not use any import statements.


In [219]:
list1 = [5,1,1,2,3,4,4,4,4,5]
         
def mystery(array):
    mylist = []
    for i in range(len(array)):
        for j in range(len(array)):
            elem1 = array[i]
            elem2 = array[j]
            if elem1 == elem2 and i != j:
                if elem2 not in mylist:
                    mylist.append(elem2)
    return mylist

print mystery(list1)

def duplicate(array):
    mydict = {}
    mylist = []

    for i in range(len(array)):
        if array[i] not in mydict:
            mydict[array[i]] = 1
        else:
            mydict[array[i]] += 1

    for k, v in mydict.items():
        if v > 1:
            mylist.append(k)

    return mylist

print duplicate(list1)

[5, 1, 4]
[1, 4, 5]


### 4. Describe the runtime of your implementation as a function of the length of the input array. Please explain your answer.

N: length of input array
Since there is no nested loop, the run time reduced to O(N)

### 5. In plain English, explain how your implementation differs from the one

I used a dictionary to count the array element count. If the count > 1, it means duplicate.
Then append duplicated element to mylist. 
The original mystery function is brute force method to iterate all possible combinations of two elements, then add duplicate element to mylist.